In [ ]:
# import Pkg
# using Pkg
# Pkg.add("FFMPEG_jll")
# Pkg.add("IterTools")
# Pkg.add("MAT")
# Pkg.add("OrderedCollections")
# Pkg.add("PlotlyJS")
# Pkg.add("Plots")
# Pkg.add("PyPlot")
# Pkg.build("FFMPEG_jll")
# Pkg.build("Plots")
# Pkg.update()
# Pkg.precompile()
# Pkg.status()
# Sys.BINDIR

In [ ]:
using Pkg
using IterTools
using LinearAlgebra
using MAT
using OrderedCollections
using Plots
using Statistics
gr()
plotlyjs()

In [ ]:
data_PA = matread("300M_PA.mat")
x = ComplexF64.(data_PA["x_in"])
x = x / maximum(abs.(x))
yr = ComplexF64.(data_PA["y_out"])
yr = yr / maximum(abs.(yr))
N = length(x)
NMSE = 10 * log10(sum(abs2, yr - x) / sum(abs2, x))
S = Int64[]
X = ComplexF64[]
println("The NMSE of Original PA data is ", NMSE, " dB.")

In [ ]:
function T(x, i, j)
    if abs(j) > 100
        error("Shift value must be within the range [-100, 100].")
    end
    x_padded = vcat(zeros(100), x, zeros(100)) # Padding 100 zeros before and after x
    mid = div(length(x), 2)
    if i == 0 # Take all of x
        y = x_padded[101-j:end-100-j]
    elseif i == 1 # Take the first half of x
        y = x_padded[101-j:100+mid-j]
    elseif i == 2 # Take the second half of x
        y = x_padded[101+mid-j:end-100-j]
    else
        error("j must be either 0, 1 or 2.")
    end
    return y
end

In [ ]:
# View the distribution characteristics of point x and point y over time
p = scatter(abs.(T(x,0,0)), abs.(T(yr,0,0)), markersize=1, color=:blue, markerstrokecolor=false, legend=false)
plot!(aspect_ratio=:equal, xlims=(0, max(1, maximum(abs.(T(x,0,0))),)), ylims=(0, max(1, maximum(abs.(T(yr,0,0))))))
title!("Original PA")
xlabel!("Normalized Input Magnitude")
ylabel!("Normalized Output Magnitude")
px = scatter(abs.(x), markersize=1, color=:blue, markerstrokecolor=false, legend=false)
title!("Input")
xlabel!("Input Sample Index")
ylabel!("Normalized Magnitude")
py = scatter(abs.(yr), markersize=1, color=:blue, markerstrokecolor=false, legend=false)
title!("Original Output")
xlabel!("Output Sample Index")
ylabel!("Normalized Magnitude")
display(p)
display(px)
display(py)

In [ ]:
# PA: Memory Polynomial model
model_name = "MP"
t = 1 # Calculation order
P = [11, 11] # 1, 3, 5, 7, 9, 11, ...11
M = [5, 5] # 1, 2, 3, 4, 5, 6, ...19
NMSE = zeros(Int((maximum(P.-1)./2 - minimum(P.-1)./2 + 1) * (maximum(M) - minimum(M) + 1)))
paramStruct = Dict()

for (K, I_) in product(round(Int, minimum((P.-1)/2)):round(Int, maximum((P.-1)/2)), minimum(M):maximum(M))
    X = zeros(ComplexF64, N, (K+1) * (I_+1))
    p = 1
    for (k, i) in product(0:K, 0:I_)
        X[:, p] = T(x,0,i) .* abs.(T(x,0,i)).^(2k)
        p += 1
    end
    c = pinv(X' * X + 1 * I(size(X, 2))) * X' * T(yr,0,0)
    yp = X * c
    NMSE[t] = 10 * log10(sum(abs2, yp - T(yr,0,0)) / sum(abs2, T(yr,0,0)))
    paramStruct[t] = OrderedDict(:c => c, :yp => yp, :NMSE => NMSE[t], :P => 2*K+1, :M => I_)
    t += 1
end
NMSE = filter(x -> x != 0, NMSE)
min_idx = argmin(NMSE)  # Get the index of the minimum value
c = paramStruct[min_idx][:c]
yp = paramStruct[min_idx][:yp]
println("The minimum NMSE in this model is ", paramStruct[min_idx][:NMSE], " dB.")
println("The length of θ matrix is ", length(c), ".")
for (key, value) in paramStruct[min_idx]
    if key ∉ [:c, :yp, :NMSE]
        println("$key: $value")
    end
end

In [ ]:
# PA: Generalized Memory Polynomial model
model_name = "GMP"
t = 1 # Calculation order
coefficients = [136] # 1, 2, 3, 4, 5, 6, ...19
Ka = [6] # 1, 2, 3, 4, 5, 6, ...19
Kb = [5] # 1, 2, 3, 4, 5, 6, ...19
Kc = [5] # 1, 2, 3, 4, 5, 6, ...19
La = [6] # 1, 2, 3, 4, 5, 6, ...3
Lb = [4] # 1, 2, 3, 4, 5, 6, ...2
Mb = [2] # 1, 2, 3, 4, 5, 6, ...1
Lc = [6] # 1, 2, 3, 4, 5, 6, ...3
Mc = [2] # 1, 2, 3, 4, 5, 6, ...2
NMSE = zeros((maximum(coefficients) - minimum(coefficients) + 1) * (maximum(Ka) - minimum(Ka) + 1) * (maximum(La) - minimum(La) + 1) * (maximum(Kb) - minimum(Kb) + 1) * (maximum(Lb) - minimum(Lb) + 1) * (maximum(Mb) - minimum(Mb) + 1) * (maximum(Kc) - minimum(Kc) + 1) * (maximum(Lc) - minimum(Lc) + 1) * (maximum(Mc) - minimum(Mc) + 1))
paramStruct = Dict()

for (coefficients, KA, LA, KB, LB, MB, KC, LC, MC) in product(minimum(coefficients):maximum(coefficients), minimum(Ka):maximum(Ka), minimum(La):maximum(La), minimum(Kb):maximum(Kb), minimum(Lb):maximum(Lb), minimum(Mb):maximum(Mb), minimum(Kc):maximum(Kc), minimum(Lc):maximum(Lc), minimum(Mc):maximum(Mc))
    # if LB + MB > 10
    #     continue
    # end
    X = zeros(ComplexF64, N, KA*LA + KB*LB*MB + KC*LC*MC)
    p = 1
    for (ka, la) in product(0:KA-1, 0:LA-1)
        X[:, p] = T(x,0,la) .* abs.(T(x,0,la)).^(2ka)
        p += 1
    end
    for (kb, lb, mb) in product(1:KB, 0:LB-1, 1:MB)
        X[:, p] = T(x,0,lb) .* abs.(T(x,0,lb+mb)).^(2kb)
        p += 1
    end
    for (kc, lc, mc) in product(1:KC, 0:LC-1, 1:MC)
        X[:, p] = T(x,0,lc) .* abs.(T(x,0,lc-mc)).^(2kc)
        p += 1
    end
    c = pinv(X' * X + 1 * I(size(X, 2))) * X' * T(yr,0,0)
    yp = X * c
    NMSE[t] = 10 * log10(sum(abs2, yp - T(yr,0,0)) / sum(abs2, T(yr,0,0)))
    paramStruct[t] = OrderedDict(:c => c, :yp => yp, :NMSE => NMSE[t], :coefficients => coefficients, :Ka => KA, :La => LA, :Kb => KB, :Lb => LB, :Mb => MB, :Kc => KC, :Lc => LC, :Mc => MC)
    t += 1
end
NMSE = filter(x -> x != 0, NMSE)
min_idx = argmin(NMSE)  # Get the index of the minimum value
c = paramStruct[min_idx][:c]
yp = paramStruct[min_idx][:yp]
println("The minimum NMSE in this model is ", paramStruct[min_idx][:NMSE], " dB.")
println("The length of θ matrix is ", length(c), ".")
for (key, value) in paramStruct[min_idx]
    if key ∉ [:c, :yp, :NMSE]
        println("$key: $value")
    end
end

In [ ]:
# PA: Vector Threshold Decomposition Generalized Memory Polynomial model
model_name = "VTD GMP"
t = 1 # Calculation order
Ka1 = [14, 14] # 1, 2, 3, 4, 5, 6, ...14
Kb1 = [11, 11] # 1, 2, 3, 4, 5, 6, ...11
Kc1 = [14, 14] # 1, 2, 3, 4, 5, 6, ...14
La1 = [3, 3] # 1, 2, 3, 4, 5, 6, ...3
Lb1 = [2, 2] # 1, 2, 3, 4, 5, 6, ...2
Mb1 = [1, 1] # 1, 2, 3, 4, 5, 6, ...1
Lc1 = [3, 3] # 1, 2, 3, 4, 5, 6, ...3
Mc1 = [2, 2] # 1, 2, 3, 4, 5, 6, ...2
Ka2 = [13, 13] # 1, 2, 3, 4, 5, 6, ...13
Kb2 = [14, 14] # 1, 2, 3, 4, 5, 6, ...14
Kc2 = [15, 15] # 1, 2, 3, 4, 5, 6, ...15
La2 = [3, 3] # 1, 2, 3, 4, 5, 6, ...3
Lb2 = [1, 1] # 1, 2, 3, 4, 5, 6, ...1
Mb2 = [2, 2] # 1, 2, 3, 4, 5, 6, ...1
Lc2 = [3, 3] # 1, 2, 3, 4, 5, 6, ...3
Mc2 = [2, 2] # 1, 2, 3, 4, 5, 6, ...2
NMSE = zeros((maximum(Ka1) - minimum(Ka1) + 1) * (maximum(La1) - minimum(La1) + 1) * (maximum(Kb1) - minimum(Kb1) + 1) * (maximum(Lb1) - minimum(Lb1) + 1) * (maximum(Mb1) - minimum(Mb1) + 1) * (maximum(Kc1) - minimum(Kc1) + 1) * (maximum(Lc1) - minimum(Lc1) + 1) * (maximum(Mc1) - minimum(Mc1) + 1) * (maximum(Ka2) - minimum(Ka2) + 1) * (maximum(La2) - minimum(La2) + 1) * (maximum(Kb2) - minimum(Kb2) + 1) * (maximum(Lb2) - minimum(Lb2) + 1) * (maximum(Mb2) - minimum(Mb2) + 1) * (maximum(Kc2) - minimum(Kc2) + 1) * (maximum(Lc2) - minimum(Lc2) + 1) * (maximum(Mc2) - minimum(Mc2) + 1))
paramStruct = Dict()

TH = [0, 0.74, 1]
xS = repeat(x, 1, length(TH)-1)
for s in 1:length(TH)-1
    xS[abs.(xS[:, s]) .<= TH[s], s] .= 0
    xS[(abs.(xS[:, s]) .<= TH[s+1]) .& (abs.(xS[:, s]) .> TH[s]), s] .= (abs.(xS[(abs.(xS[:, s]) .<= TH[s+1]) .& (abs.(xS[:, s]) .> TH[s]), s]) .- TH[s]) .* exp.(1im * angle.(xS[(abs.(xS[:, s]) .<= TH[s+1]) .& (abs.(xS[:, s]) .> TH[s]), s]))
    xS[abs.(xS[:, s]) .> TH[s+1], s] .= (TH[s+1] - TH[s]) .* exp.(1im * angle.(xS[abs.(xS[:, s]) .> TH[s+1], s]))
end

for (KA1, LA1, KB1, LB1, MB1, KC1, LC1, MC1, KA2, LA2, KB2, LB2, MB2, KC2, LC2, MC2) in product(minimum(Ka1):maximum(Ka1), minimum(La1):maximum(La1), minimum(Kb1):maximum(Kb1), minimum(Lb1):maximum(Lb1), minimum(Mb1):maximum(Mb1), minimum(Kc1):maximum(Kc1), minimum(Lc1):maximum(Lc1), minimum(Mc1):maximum(Mc1), minimum(Ka2):maximum(Ka2), minimum(La2):maximum(La2), minimum(Kb2):maximum(Kb2), minimum(Lb2):maximum(Lb2), minimum(Mb2):maximum(Mb2), minimum(Kc2):maximum(Kc2), minimum(Lc2):maximum(Lc2), minimum(Mc2):maximum(Mc2))
    if LB1 + MB1 > 3 || LB2 + MB2 > 3
        continue
    end
    X = zeros(ComplexF64, N, KA1*LA1 + KB1*LB1*MB1 + KC1*LC1*MC1 + KA2*LA2 + KB2*LB2*MB2 + KC2*LC2*MC2)
    p = 1
    for (ka, la) in product(0:KA1-1, 0:LA1-1)
        X[:, p] = T(xS[:, 1],0,la) .* abs.(T(xS[:, 1],0,la)).^(2ka)
        p += 1
    end
    for (kb, lb, mb) in product(1:KB1, 0:LB1-1, 1:MB1)
        X[:, p] = T(xS[:, 1],0,lb) .* abs.(T(xS[:, 1],0,lb+mb)).^(2kb)
        p += 1
    end
    for (kc, lc, mc) in product(1:KC1, 0:LC1-1, 1:MC1)
        X[:, p] = T(xS[:, 1],0,lc) .* abs.(T(xS[:, 1],0,lc-mc)).^(2kc)
        p += 1
    end
    for (ka, la) in product(0:KA2-1, 0:LA2-1)
        X[:, p] = T(xS[:, 2],0,la) .* abs.(T(xS[:, 2],0,la)).^(2ka)
        p += 1
    end
    for (kb, lb, mb) in product(1:KB2, 0:LB2-1, 1:MB2)
        X[:, p] = T(xS[:, 2],0,lb) .* abs.(T(xS[:, 2],0,lb+mb)).^(2kb)
        p += 1
    end
    for (kc, lc, mc) in product(1:KC2, 0:LC2-1, 1:MC2)
        X[:, p] = T(xS[:, 2],0,lc) .* abs.(T(xS[:, 2],0,lc-mc)).^(2kc)
        p += 1
    end
    c = pinv(X' * X + 1 * I(size(X, 2))) * X' * T(yr,0,0)
    yp = X * c
    NMSE[t] = 10 * log10(sum(abs2, yp - T(yr,0,0)) / sum(abs2, T(yr,0,0)))
    paramStruct[t] = OrderedDict(:c => c, :yp => yp, :NMSE => NMSE[t], :Ka1 => KA1, :La1 => LA1, :Kb1 => KB1, :Lb1 => LB1, :Mb1 => MB1, :Kc1 => KC1, :Lc1 => LC1, :Mc1 => MC1, :Ka2 => KA2, :La2 => LA2, :Kb2 => KB2, :Lb2 => LB2, :Mb2 => MB2, :Kc2 => KC2, :Lc2 => LC2, :Mc2 => MC2)
    t += 1
end
NMSE = filter(x -> x != 0, NMSE)
min_idx = argmin(NMSE)  # Get the index of the minimum value
c = paramStruct[min_idx][:c]
yp = paramStruct[min_idx][:yp]
println("The minimum NMSE in this model is ", paramStruct[min_idx][:NMSE], " dB.")
println("The length of θ matrix is ", length(c), ".")
for (key, value) in paramStruct[min_idx]
    if key ∉ [:c, :yp, :NMSE]
        println("$key: $value")
    end
end

In [ ]:
# PA: Dynamic deviation reduction (DDR)-based Volterra model
model_name = "DDR"
t = 1 # Calculation order
P = [75, 75] # 3, 5, 7, 9, 11, 13, ...75
M = [2, 2] # 1, 2, 3, 4, 5, 6, ...2
NMSE = zeros(Int((maximum(P.-1)./2 - minimum(P.-1)./2 + 1) * (maximum(M) - minimum(M) + 1)))
paramStruct = Dict()

for (K, I) in product(round(Int, minimum((P.-1)/2)):round(Int, maximum((P.-1)/2)), minimum(M):maximum(M))
    X = zeros(ComplexF64, N, (K+1) * (I+1) + K * I)
    p = 1 # Position of the specific column to be filled
    for (k, i) in product(0:K, 0:I)
        X[:, p] = T(x,0,i) .* abs.(T(x,0,0)).^(2k)
        p += 1
    end
    for (k, i) in product(1:K, 1:I)
        X[:, p] = conj(T(x,0,i)) .* T(x,0,0).^2 .* abs.(T(x,0,0)).^(2(k-1))
        p += 1
    end
    c = pinv(X' * X + 1 * I(size(X, 2))) * X' * T(yr,0,0)
    yp = X * c
    NMSE[t] = 10 * log10(sum(abs2, yp - T(yr,0,0)) / sum(abs2, T(yr,0,0)))
    paramStruct[t] = OrderedDict(:c => c, :yp => yp, :NMSE => NMSE[t], :P => 2*K+1, :M => I)
    t += 1
end


In [ ]:
# PA: Vector Threshold Decomposition Dynamic deviation reduction (DDR)-based Volterra model
model_name = "VTD DDR"
t = 1 # Calculation order
P1 = [21, 21] # 3, 5, 7, 9, 11, 13, ...21
M1 = [2, 2] # 1, 2, 3, 4, 5, 6, ...2
P2 = [33, 33] # 3, 5, 7, 9, 11, 13, ...33
M2 = [2, 2] # 1, 2, 3, 4, 5, 6, ...2
NMSE = zeros(Int((maximum(P1.-1)./2 - minimum(P1.-1)./2 + 1) * (maximum(M1) - minimum(M1) + 1) * (maximum(P2.-1)./2 - minimum(P2.-1)./2 + 1) * (maximum(M2) - minimum(M2) + 1)))
paramStruct = Dict()

TH = [0, 0.74, 1]
xS = repeat(x, 1, length(TH)-1)
for s in 1:length(TH)-1
    xS[abs.(xS[:, s]) .<= TH[s], s] .= 0
    xS[(abs.(xS[:, s]) .<= TH[s+1]) .& (abs.(xS[:, s]) .> TH[s]), s] .= (abs.(xS[(abs.(xS[:, s]) .<= TH[s+1]) .& (abs.(xS[:, s]) .> TH[s]), s]) .- TH[s]) .* exp.(1im * angle.(xS[(abs.(xS[:, s]) .<= TH[s+1]) .& (abs.(xS[:, s]) .> TH[s]), s]))
    xS[abs.(xS[:, s]) .> TH[s+1], s] .= (TH[s+1] - TH[s]) .* exp.(1im * angle.(xS[abs.(xS[:, s]) .> TH[s+1], s]))
end

for (K1, I1, K2, I2) in product(round(Int, minimum((P1.-1)/2)):round(Int, maximum((P1.-1)/2)), minimum(M1):maximum(M1), round(Int, minimum((P2.-1)/2)):round(Int, maximum((P2.-1)/2)), minimum(M2):maximum(M2))
    X = zeros(ComplexF64, N, (K1+1) * (I1+1) + K1 * I1 + (K2+1) * (I2+1) + K2 * I2)
    p = 1 # Position of the specific column to be filled
    for (k, i) in product(0:K1, 0:I1)
        X[:, p] = T(xS[:, 1],0,i) .* abs.(T(xS[:, 1],0,0)).^(2k)
        p += 1
    end
    for (k, i) in product(1:K1, 1:I1)
        X[:, p] = conj(T(xS[:, 1],0,i)) .* T(xS[:, 1],0,0).^2 .* abs.(T(xS[:, 1],0,0)).^(2(k-1))
        p += 1
    end
    for (k, i) in product(0:K2, 0:I2)
        X[:, p] = T(xS[:, 2],0,i) .* abs.(T(xS[:, 2],0,0)).^(2k)
        p += 1
    end
    for (k, i) in product(1:K2, 1:I2)
        X[:, p] = conj(T(xS[:, 2],0,i)) .* T(xS[:, 2],0,0).^2 .* abs.(T(xS[:, 2],0,0)).^(2(k-1))
        p += 1
    end
    c = pinv(X' * X + 1 * I(size(X, 2))) * X' * T(yr,0,0)
    yp = X * c
    NMSE[t] = 10 * log10(sum(abs2, yp - T(yr,0,0)) / sum(abs2, T(yr,0,0)))
    paramStruct[t] = OrderedDict(:c => c, :yp => yp, :NMSE => NMSE[t], :P1 => 2*K1+1, :M1 => I1, :P2 => 2*K2+1, :M2 => I2)
    t += 1
end

In [ ]:
# PA: Simplified 2nd-order DDR Model
model_name = "2nd DDR"
t = 1 # Calculation order
P = [5, 5] # 3, 5, 7, 9, 11, 13, ...49
M = [3, 3] # 1, 2, 3, 4, 5, 6, ...2
NMSE = zeros(Int((maximum(P.-1)./2 - minimum(P.-1)./2 + 1) * (maximum(M) - minimum(M) + 1)))
paramStruct = Dict()

for (K, I) in product(round(Int, minimum((P.-1)/2)):round(Int, maximum((P.-1)/2)), minimum(M):maximum(M))
    X = zeros(ComplexF64, N, (K+1) * (I+1) + 3 * K * I)
    p = 1 # Position of the specific column to be filled
    for (k, i) in product(0:K, 0:I)
        X[:, p] = T(x,0,i) .* abs.(T(x,0,0)).^(2k)
        p += 1
    end
    for (k, i) in product(1:K, 1:I)
        X[:, p] = conj(T(x,0,i)) .* T(x,0,0).^2 .* abs.(T(x,0,0)).^(2(k-1))
        p += 1
    end
    for (k, i) in product(1:K, 1:I)
        X[:, p] = abs.(T(x,0,i)).^2 .* T(x,0,0) .* abs.(T(x,0,0)).^(2(k-1))
        p += 1
    end
    for (k, i) in product(1:K, 1:I)
        X[:, p] = T(x,0,i).^2 .* conj(T(x,0,0)) .* abs.(T(x,0,0)).^(2(k-1))
        p += 1
    end
    c = pinv(X' * X + 1 * I(size(X, 2))) * X' * T(yr,0,0)
    yp = X * c
    NMSE[t] = 10 * log10(sum(abs2, yp - T(yr,0,0)) / sum(abs2, T(yr,0,0)))
    paramStruct[t] = OrderedDict(:c => c, :yp => yp, :NMSE => NMSE[t], :P => 2*K+1, :M => I)
    t += 1
end
NMSE = filter(x -> x != 0, NMSE)
min_idx = argmin(NMSE)  # Get the index of the minimum value
c = paramStruct[min_idx][:c]
yp = paramStruct[min_idx][:yp]
println("The minimum NMSE in this model is ", paramStruct[min_idx][:NMSE], " dB.")
println("The length of θ matrix is ", length(c), ".")
for (key, value) in paramStruct[min_idx]
    if key ∉ [:c, :yp, :NMSE]
        println("$key: $value")
    end
end

In [ ]:
# PA: Vector Threshold Decomposition Simplified 2nd-order DDR Model
model_name = "VTD 2nd DDR"
t = 1 # Calculation order
P1 = [21, 21] # 3, 5, 7, 9, 11, 13, ...21
M1 = [2, 2] # 1, 2, 3, 4, 5, 6, ...2
P2 = [49, 49] # 3, 5, 7, 9, 11, 13, ...49
M2 = [2, 2] # 1, 2, 3, 4, 5, 6, ...2
NMSE = zeros(Int((maximum(P1.-1)./2 - minimum(P1.-1)./2 + 1) * (maximum(M1) - minimum(M1) + 1) * (maximum(P2.-1)./2 - minimum(P2.-1)./2 + 1) * (maximum(M2) - minimum(M2) + 1)))
paramStruct = Dict()

TH = [0, 0.74, 1]
xS = repeat(x, 1, length(TH)-1)
for s in 1:length(TH)-1
    xS[abs.(xS[:, s]) .<= TH[s], s] .= 0
    xS[(abs.(xS[:, s]) .<= TH[s+1]) .& (abs.(xS[:, s]) .> TH[s]), s] .= (abs.(xS[(abs.(xS[:, s]) .<= TH[s+1]) .& (abs.(xS[:, s]) .> TH[s]), s]) .- TH[s]) .* exp.(1im * angle.(xS[(abs.(xS[:, s]) .<= TH[s+1]) .& (abs.(xS[:, s]) .> TH[s]), s]))
    xS[abs.(xS[:, s]) .> TH[s+1], s] .= (TH[s+1] - TH[s]) .* exp.(1im * angle.(xS[abs.(xS[:, s]) .> TH[s+1], s]))
end

for (K1, I1, K2, I2) in product(round(Int, minimum((P1.-1)/2)):round(Int, maximum((P1.-1)/2)), minimum(M1):maximum(M1), round(Int, minimum((P2.-1)/2)):round(Int, maximum((P2.-1)/2)), minimum(M2):maximum(M2))
    X = zeros(ComplexF64, N, (K1+1) * (I1+1) + 3 * K1 * I1 + (K2+1) * (I2+1) + 3 * K2 * I2)
    p = 1 # Position of the specific column to be filled
    for (k, i) in product(0:K1, 0:I1)
        X[:, p] = T(xS[:, 1],0,i) .* abs.(T(xS[:, 1],0,0)).^(2k)
        p += 1
    end
    for (k, i) in product(1:K1, 1:I1)
        X[:, p] = conj(T(xS[:, 1],0,i)) .* T(xS[:, 1],0,0).^2 .* abs.(T(xS[:, 1],0,0)).^(2(k-1))
        p += 1
    end
    for (k, i) in product(1:K1, 1:I1)
        X[:, p] = abs.(T(xS[:, 1],0,i)).^2 .* T(xS[:, 1],0,0) .* abs.(T(xS[:, 1],0,0)).^(2(k-1))
        p += 1
    end
    for (k, i) in product(1:K1, 1:I1)
        X[:, p] = T(xS[:, 1],0,i).^2 .* conj(T(xS[:, 1],0,0)) .* abs.(T(xS[:, 1],0,0)).^(2(k-1))
        p += 1
    end
    for (k, i) in product(0:K2, 0:I2)
        X[:, p] = T(xS[:, 2],0,i) .* abs.(T(xS[:, 2],0,0)).^(2k)
        p += 1
    end
    for (k, i) in product(1:K2, 1:I2)
        X[:, p] = conj(T(xS[:, 2],0,i)) .* T(xS[:, 2],0,0).^2 .* abs.(T(xS[:, 2],0,0)).^(2(k-1))
        p += 1
    end
    for (k, i) in product(1:K2, 1:I2)
        X[:, p] = abs.(T(xS[:, 2],0,i)).^2 .* T(xS[:, 2],0,0) .* abs.(T(xS[:, 2],0,0)).^(2(k-1))
        p += 1
    end
    for (k, i) in product(1:K2, 1:I2)
        X[:, p] = T(xS[:, 2],0,i).^2 .* conj(T(xS[:, 2],0,0)) .* abs.(T(xS[:, 2],0,0)).^(2(k-1))
        p += 1
    end
    c = pinv(X' * X + 1 * I(size(X, 2))) * X' * T(yr,0,0)
    yp = X * c
    NMSE[t] = 10 * log10(sum(abs2, yp - T(yr,0,0)) / sum(abs2, T(yr,0,0)))
    paramStruct[t] = OrderedDict(:c => c, :yp => yp, :NMSE => NMSE[t], :P1 => 2*K1+1, :M1 => I1, :P2 => 2*K2+1, :M2 => I2)
    t += 1
end
NMSE = filter(x -> x != 0, NMSE)
min_idx = argmin(NMSE)  # Get the index of the minimum value
c = paramStruct[min_idx][:c]
yp = paramStruct[min_idx][:yp]
println("The minimum NMSE in this model is ", paramStruct[min_idx][:NMSE], " dB.")
println("The length of θ matrix is ", length(c), ".")
for (key, value) in paramStruct[min_idx]
    if key ∉ [:c, :yp, :NMSE]
        println("$key: $value")
    end
end

In [ ]:
# Plot and Print
NMSE = filter(x -> x != 0, NMSE)
min_idx = argmin(NMSE)  # Get the index of the minimum value
c = paramStruct[min_idx][:c]
yp = paramStruct[min_idx][:yp]

# Figure 1
p1 = scatter(abs.(T(x,0,0)), abs.(T(yr,0,0)), markersize=1, color=:blue, markerstrokecolor=false, legend=false)
scatter!(abs.(T(yr,0,0)), abs.(yp), markersize=1, color=:green, markerstrokecolor=false, legend=false)
plot!(aspect_ratio=:equal, xlims=(0, max(1, maximum(abs.(T(x,0,0))), maximum(abs.(T(yr,0,0))))), ylims=(0, max(1, maximum(abs.(yp)), maximum(abs.(T(yr,0,0))))))
title!("PA $(model_name)")
xlabel!("Normalized Magnitude")
ylabel!("Normalized Magnitude")

# Figure 2
p2 = scatter(abs.(T(x,0,0)), rad2deg.(angle.(T(yr,0,0)./T(x,0,0))), markersize=1, color=:blue, markerstrokecolor=false, legend=false)
scatter!(abs.(T(yr,0,0)), rad2deg.(angle.(yp./T(yr,0,0))), markersize=1, color=:green, markerstrokecolor=false, legend=false)
plot!(ylims=(-180, 180), yticks=-180:60:180)
title!("PA $(model_name)")
xlabel!("Normalized Magnitude")
ylabel!("Phase Difference (Degrees)")

# Figure 3
p3 = scatter(NMSE, markersize=1, color=:blue, markerstrokecolor=false, legend=false)
scatter!([min_idx], [paramStruct[min_idx][:NMSE]], markersize=2, color=:red, markerstrokecolor=false, legend=false)
title!("NMSE Comparison")
xlabel!("Calculation order of NMSE")
ylabel!("NMSE (dB)")

# Display plots
display(p1)
display(p2)
display(p3)

# Print specific information
println("The minimum NMSE in this model is ", paramStruct[min_idx][:NMSE], " dB.")
println("The length of θ matrix is ", length(c), ".")
for (key, value) in paramStruct[min_idx]
    if key ∉ [:c, :yp, :NMSE]
        println("$key: $value")
    end
end

data_c = matopen("c.mat", "w") 
write(data_c, "c", c)
close(data_c)

In [ ]:
# Observe information at a specific point
for (key, value) in paramStruct[1]
    if key ∉ [:yp, :c]
        println("$key: $value")
    end
end